Reading in the data and then filtering out SNPs which do not meet p value significance before joining the data together.

In [1]:
%load_ext autoreload
%autoreload 2

import polars as pl

# exp: Exposure
# out: Outcome
# ea: Exposure allele
# oa: Other allele

exp_header_dict = {
  'rsID':'rsid',
  'CHROM':'chr_exp',
  'ALT':'ea_exp',
  'REF':'oa_exp',
  'POOLED_ALT_AF':'eaf_exp',
  'EFFECT_SIZE':'beta_exp',
  'SE':'se_exp',
  'pvalue':'pval_exp'
}

out_header_dict = {
  'markername':'rsid',
  'chr':'chr_out',
  'bp_hg19':'pos_out',
  'effect_allele':'ea_out',
  'noneffect_allele':'oa_out',
  'effect_allele_freq':'eaf_out',
  'beta':'beta_out',
  'se_dgc':'se_out',
  'p_dgc':'pval_out'}

pthresh = 5e-8

# Renaming columns and filtering data to only include observations which fulfill significance threshold
dexp = (pl.scan_csv("dataset/ldlc_gwas.txt",separator="\t")
        .rename(exp_header_dict)
        .filter((pl.col('pval_exp') < pthresh)))
dout = (pl.scan_csv("dataset/mi_gwas.tsv",separator="\t")
        .rename(out_header_dict))

combined = (dexp.join(dout, on='rsid')
            # Convert all data to lowercase
            .with_columns(
                pl.col('ea_exp').str.to_lowercase(),
                pl.col('oa_exp').str.to_lowercase(),
                pl.col('ea_out').str.to_lowercase(),
                pl.col('oa_out').str.to_lowercase(),
            # Convert minor allele freq to effect allele freq
                pl.col('eaf_exp').mul(-1).add(1))
            .collect())

print(combined.shape)

(75089, 26)


We can use the provided harmonize function.

In [2]:
from MR.harmonize import harmonize

total = harmonize(combined, palindromic_action=1, palindromic_threshold=0.08)
print(total.shape)

# Seeing if duplicates are dropped
print(total.unique(subset=['rsid']).shape)

(73095, 26)
(73095, 26)


Clumping the data based on Linkage Disequilibirum (LD). This is to filter out SNPs which are closely correlated in order to essentially avoid double counting results into the final calculation.

In [3]:
from MR.ld import ld_clump

pruned_rsids = ld_clump(total['rsid'], total['pval_exp'])

processed_data = (total.join(pruned_rsids, on='rsid'))

print(processed_data.shape)

(379, 26)


In [4]:
from MR.ld import ld_matrix

# print(ld_matrix(processed_data['rsid']))

Calculating causal effects.

In [73]:
from MR.mr_methods.egger_regression import mr_egger_regression
from MR.mr_methods.inverse_variance_weighted import mr_inverse_variance_weighted
from MR.mr_methods.maximum_likelihood import mr_maximum_likelihood
from MR.mr_methods.median import mr_penalised_weighted_median, mr_simple_median, mr_weighted_median
from MR.mr_methods.mode import mr_simple_mode, mr_weighted_mode
from MR.mr_methods.outliers import mr_presso_global_test, mr_remove_outliers
from MR.mr_methods.wald_ratio import mr_wald_ratio


data = mr_remove_outliers(processed_data, 'cooks distance')
# data = processed_data

mr_presso_global_test(data, k=1000)

# print('Inverse Variance Weighted')
# result = mr_inverse_variance_weighted(data['beta_exp'], data['beta_out'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Wald ratio')
# result = mr_wald_ratio(data['beta_exp'], data['beta_out'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Simple Median')
# result = mr_simple_median(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'], 1000)
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Weighted Median')
# result = mr_weighted_median(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# # print('Penalised Weighted Median')
# # result = mr_penalised_weighted_median(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# # print(f'Effect: {result["effect"]}')
# # print(f'se: {result["se"]}')
# # print(f'pval: {result["pval"]}\n')

# print('Egger Regression')
# result = mr_egger_regression(data['beta_exp'], data['beta_out'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Simple Mode')
# result = mr_simple_mode(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# print('Weighted Mode')
# result = mr_weighted_mode(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')

# # print('Penalised Weighted Mode')
# # result = mr_penalised_weighted_mode(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# # print(f'Effect: {result["effect"]}')
# # print(f'se: {result["se"]}')
# # print(f'pval: {result["pval"]}\n')

# print('Maximum Likelihood')
# result = mr_maximum_likelihood(data['beta_exp'], data['beta_out'], data['se_exp'], data['se_out'])
# print(f'Effect: {result["effect"]}')
# print(f'se: {result["se"]}')
# print(f'pval: {result["pval"]}\n')


0.5139308488129113
[0.20522607822943922, 0.1310994851233399, 0.14558586540005944, 0.19777087564123394, 0.1539444697245629, 0.19200202430903507, 0.14053813393812817, 0.17966447012741196, 0.13216650604324254, 0.17036368030172386, 0.1609671179225337, 0.165198793251735, 0.14318381775310465, 0.12824191198631446, 0.14228720141594695, 0.15405645090797904, 0.15165468216874556, 0.12457332980739408, 0.14444450839537135, 0.15313143120458222, 0.1500851844145176, 0.1632497418620656, 0.12578369084714394, 0.18155550809608326, 0.13222603674559394, 0.14926849587551486, 0.16612391732285578, 0.12461042362714103, 0.15347689954521138, 0.18577476314754376, 0.13794577295727176, 0.1750210869821127, 0.17932385326446498, 0.17024391921210208, 0.18056767814522878, 0.1649002271620198, 0.14177494841862887, 0.1575647624064084, 0.15749234197301173, 0.14580052934853277, 0.13637290333797727, 0.18806377860609969, 0.13468687468639298, 0.19477992785076217, 0.18464038072638117, 0.16457402177004932, 0.1576710186520632, 0.17